In [4]:
import pandas as pd
import numpy as np

In [2]:
df_data = pd.read_parquet("datos/chb02_raw_eeg_128.parquet")
df_annotations = pd.read_excel("datos/df_annotation_full.xlsx")

In [5]:
df_data["id"] = df_data.groupby("filename").cumcount()

In [6]:
def seizure_id(line):
    F = 64
    S = 30
    if line["type"] == "normal":
        return 0
    for a in df_annotations[df_annotations["filename"] == line["filename"]].values:
        if a[5]*F<=line["id"] and a[6]*F>=line["id"]: # Seizure
            return 1
        if a[5]*F > line["id"] and a[5]*F - F*S <= line["id"]: # Antes Seizure
            return 2
        if a[6]*F  + F*S >= line["id"] and a[6]*F < line["id"]: # Despues Seizure
            return 3
    return 0

In [7]:
df_data["tag"] = df_data.apply(seizure_id, axis=1)

In [9]:
df_data[df_data["tag"]==3]

,FP1-F7,F7-T7,T7-P7,P7-O1,FP1-F3,F3-C3,C3-P3,P3-O1,FP2-F4,F4-C4,...,FZ-CZ,CZ-PZ,T7-FT9,FT9-FT10,FT10-T8,type,PatID,filename,id,tag
14605572,-110.378510,199.462753,48.644688,-31.452991,-29.890110,237.753357,-144.761902,44.346764,-61.929180,-4.884005,...,84.200241,-120.927963,-53.333332,136.166061,-25.201466,seizure,chb02,chb02_16+.edf,195393,3
14605576,-92.796089,201.416367,9.963370,-33.015873,-8.791209,151.794876,-65.836388,9.181930,-31.843712,-20.512821,...,128.351654,-132.649567,-101.001221,101.001221,-21.684982,seizure,chb02,chb02_16+.edf,195394,3
14605580,-113.894997,124.835167,93.577530,-61.538460,-2.148962,92.405373,-69.743591,22.075703,1.367521,-52.161171,...,67.008545,-186.178268,-71.697189,125.616608,-27.155067,seizure,chb02,chb02_16+.edf,195395,3
14605584,-118.192917,152.576309,30.671551,-100.219780,22.466423,33.406593,-115.067154,23.247864,19.731380,-89.670326,...,-6.837607,-287.374847,-114.676437,87.716728,-26.764347,seizure,chb02,chb02_16+.edf,195396,3
14605588,-122.881561,66.227104,38.876678,-167.423691,43.565323,-27.155067,-127.570206,-72.478630,18.559219,-146.324783,...,-124.835167,-275.262512,-89.279610,124.053726,-51.379730,seizure,chb02,chb02_16+.edf,195397,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14807536,-57.631256,-3.711844,2.930403,58.021976,1.367521,63.492062,-3.321123,-62.319901,17.387058,13.479854,...,36.532356,31.843712,43.565323,81.855919,3.321123,seizure,chb02,chb02_16.edf,15484,3
14807540,-30.671551,10.354091,-26.764347,47.472527,19.731380,74.822952,-49.816849,-44.346764,35.360195,0.195360,...,32.625153,49.035408,23.247864,66.227104,-11.526252,seizure,chb02,chb02_16.edf,15485,3
14807544,-31.452991,-9.963370,-0.195360,48.253967,20.122101,72.087914,-46.300365,-41.611721,34.578754,-9.572650,...,25.201466,49.816849,38.095238,54.896214,23.247864,seizure,chb02,chb02_16.edf,15486,3
14807548,-17.777779,16.996338,-21.294262,36.141636,29.499390,77.167274,-40.439560,-52.942612,29.108669,-11.916972,...,36.923077,45.128204,11.916972,82.637360,-16.214897,seizure,chb02,chb02_16.edf,15487,3


In [ ]:
df_data.to_parquet("chb02_raw_eeg_64_tag.parquet")

In [5]:
df_data = pd.read_parquet("Datos_procesados/chb02_raw_eeg_128.parquet")

In [4]:
df_data.shape[0]

4062689

In [6]:
F = 64
Division_window = 32
windows = []
metadatos = []
for file in df_data["filename"].unique():
    df_file = df_data[df_data["filename"] == file]
    for tag in [0, 1, 2, 3]:
        df_tag = df_file[df_file["tag"] == tag].drop(["type", "PatID", "filename", "id", "tag"], axis=1)
        df_tag = df_tag.drop(df_tag.tail(df_tag.shape[0]%F).index)
        if df_tag.shape[0]>0:
            windows = windows + np.array_split(df_tag, df_tag.shape[0]/F)
            metadatos = metadatos + [[file, tag] for _ in range(int(df_tag.shape[0]/F))]
            if tag == 1:
                for i in range(Division_window):
                    df_tag = np.roll(df_tag, int(-F/Division_window), axis=0)
                    windows = windows + np.array_split(df_tag, df_tag.shape[0]/F)
                    metadatos = metadatos + [[file, tag] for _ in range(int(df_tag.shape[0]/F))]

In [76]:
count_1 = 0
count_0 = 0
for file, i in metadatos:
    if i==1:
        count_1 += 1
    elif i==0:
        count_0 += 1
    
print(count_1)
print(count_0)
print(count_1/count_0)
print(count_0/count_1)

5379
63194
0.08511884039624015
11.748280349507343


In [94]:
np.savez("Datos_procesados/", windows = windows)

In [95]:
test = np.load("Datos_procesados/metadata.npz")

In [1]:
import os

In [10]:
metadatos = pd.DataFrame(metadatos, columns=["filename", "tag"])
metadatos.to_parquet(os.path.join("Datos_procesados", file+"_metadatos"))


In [11]:
metadatos.to_parquet(os.path.join("Datos_procesados", file+"_metadatos"))


In [13]:
metadatos

,filename,tag
0,chb02_01.edf,0
1,chb02_01.edf,0
2,chb02_01.edf,0
3,chb02_01.edf,0
4,chb02_01.edf,0
...,...,...
68688,chb02_18.edf,0
68689,chb02_18.edf,0
68690,chb02_18.edf,0
68691,chb02_18.edf,0


In [15]:
windows[0]

,FP1-F7,F7-T7,T7-P7,P7-O1,FP1-F3,F3-C3,C3-P3,P3-O1,FP2-F4,F4-C4,...,P4-O2,FP2-F8,F8-T8,T8-P8,P8-O2,FZ-CZ,CZ-PZ,T7-FT9,FT9-FT10,FT10-T8
0,25.982906,5.274725,-52.551891,-19.731380,26.764347,-12.698413,-84.590965,29.499390,-27.155067,-49.816849,...,-0.195360,-53.724052,-72.087914,9.963370,31.062271,-44.346764,-71.697189,-18.559219,-59.975578,-75.995117
4,0.195360,0.195360,0.195360,-0.195360,0.195360,0.195360,0.195360,-0.195360,0.195360,0.195360,...,-0.195360,0.195360,0.195360,0.195360,-0.195360,0.195360,-0.195360,0.586081,-0.976801,0.976801
8,-0.195360,-1.758242,3.321123,-0.976801,-0.586081,0.195360,0.976801,-0.586081,0.195360,0.586081,...,-0.976801,0.586081,0.586081,0.195360,-0.976801,0.195360,-0.195360,8.791209,-2.148962,-2.148962
12,6.056166,2.148962,-11.916972,-6.056166,8.009768,-2.539683,-9.963370,-4.102564,-3.711844,0.586081,...,-4.884005,1.367521,-6.446887,1.367521,-4.493285,-12.307693,6.446887,-29.108669,0.976801,14.652015
16,2.539683,-13.870574,-13.870574,-0.976801,-0.195360,-13.089133,-7.228327,-4.493285,-10.744811,19.731380,...,-8.009768,4.493285,4.493285,7.619048,-9.963370,-17.387058,28.327229,-4.102564,-1.367521,16.996338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236,-5.274725,-17.777779,25.201466,-71.306473,6.837607,-25.201466,-70.134308,18.168499,-19.340660,-26.373627,...,30.280830,-20.122101,-7.228327,-16.605618,-13.479854,-42.002441,-69.352867,12.307693,-11.916972,-11.135531
240,-11.916972,-14.652015,-6.446887,-66.227104,-4.493285,-42.783882,-83.028084,29.108669,-10.744811,-37.313797,...,24.810745,-13.089133,-18.949940,-31.062271,-22.075703,-50.207569,-94.749695,25.592186,-15.824176,-27.155067
244,-14.261294,-12.698413,-33.015873,-45.909645,-2.539683,-52.161171,-84.590965,31.843712,-16.214897,-27.936508,...,18.949940,-14.261294,-17.387058,-31.452991,-26.373627,-40.439560,-113.504272,24.029305,-0.586081,-27.936508
248,-18.168499,-39.267399,-29.108669,-26.764347,-3.321123,-54.505493,-91.233208,33.797314,-13.479854,-25.982906,...,5.274725,-25.982906,-13.479854,-29.108669,-27.936508,-38.485958,-130.305252,35.360195,11.135531,-27.936508


In [16]:
filename = "chb02_raw_eeg_128.parquet"
filename[:filename.find(".")]

'chb02_raw_eeg_128'

In [25]:
windows = np.load("Datos_procesados/chb02_raw_eeg_128_windows.npz")

In [65]:
from math import log, e

power_energy = sum(algo**2)

st.entropy

In [68]:
metrics_windows = []
for window in windows["windows"]:
    metrics = window.aggregate(["mean", "std", "min", "max", "median", "kurtosis", "skew"])
    metrics = metrics.transpose()
    metrics["power_energy"] = (window**2).sum()
    #Siempre es igual, ahora mismo no sirve de nada
    #vc = window.value_counts(normalize=True, sort=False)
    #base = e
    #metrics["entropy"] = -(vc * np.log(vc)/np.log(base)).sum()
    metrics_windows.append(metrics)

KeyboardInterrupt: 

In [ ]:
metrics_windows[0]